In [8]:
!pip install torch==1.4.0
!pip install transformers==2.9.0
!pip install pytorch_lightning==0.7.5

ERROR: Could not find a version that satisfies the requirement torch==1.4.0 (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2)
ERROR: No matching distribution found for torch==1.4.0


In [1]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer


def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

device  cuda


In [11]:
import pickle

In [5]:
import joblib

joblib.dump(model,  'pickle_file_name.pkl',compress=9)


['pickle_file_name.pkl']

In [12]:
pickle.dump(model, open('model.pkl','wb'))

In [13]:
model1 = pickle.load(open('model.pkl','rb'))

In [2]:
import pandas as pd
df=pd.read_csv("questions- alex.csv")

In [3]:
df['Questions '][3]="whats is your name? | where do you live ?"

In [46]:
def generator(sentence):
    text =  "paraphrase: " + sentence + " </s>"

    encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
    beam_outputs = model.generate(
        input_ids=input_ids, attention_mask=attention_masks,
        do_sample=True,
        max_length=256,
        top_k=120,
        top_p=0.98,
        early_stopping=True,
        num_return_sequences=10
    )

#     print ("\nOriginal Question ::")
#     print (sentence)
#     print ("\n")
#     print ("Paraphrased Questions :: ")
    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            final_outputs.append(sent)
    
    return final_outputs
    

In [47]:
import re

def findmatches(pattern, phrase):

    for p in patterns:
        match= re.findall(p, phrase)
    return match[0]

patterns= ['[^!.?]+']



In [53]:
import csv

for i in df['Questions ']:
    final_outputs=[]
    if "|" in i:
        x=i.split('|')
        for j in x:
            final_outputs.append(generator(j))
    
        final_outputs = [item for sublist in final_outputs for item in sublist]
        output={}
        for c, final_output in enumerate(final_outputs):
            output[c]=final_output  
        filename=findmatches(patterns, i)
        with open("%s.csv"%filename, 'w') as csv_file:  
            writer = csv.writer(csv_file)
            for key, value in output.items():
                writer.writerow([key, value])
    else:           
        final_outputs=generator(i) 
       
        output={}
        for c, final_output in enumerate(final_outputs):
            output[c]=final_output  
        
        filename=findmatches(patterns, i)
        with open("%s.csv"%filename, 'w') as csv_file:  
            writer = csv.writer(csv_file)
            for key, value in output.items():
                writer.writerow([key, value])
        


    

In [55]:
df1 = pd.read_csv('are you guys data driven firm .csv')

In [56]:
df1

,0,Are you guys Data driven firm?
0,1,"Are you guys data driven firm? If yes, what do..."
1,2,Are you guys data driven firm?
2,3,Are you guys data driven startup?
3,4,Are you guys data driven company?
4,5,Are you guys data driven business?
5,6,Are you guys a data driven?


In [54]:
with open('are you guys data driven firm .csv') as csv_file:
    reader = csv.reader(csv_file)
    mydict = dict(reader)

ValueError: dictionary update sequence element #1 has length 0; 2 is required

In [1]:

# importing required libraries
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split